In [4]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px
import seaborn as sns
from plotly.offline import iplot,plot
from plotly.subplots import make_subplots
from warnings import filterwarnings
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import r2_score,mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Disable filter warning
filterwarnings('ignore')

In [6]:
# Read dataframe
laptop_data = pd.read_csv('Laptops.csv')
laptop_data.head(5)

,Unnamed: 0,Brand,Model Name,Processor,Operating System,Storage,RAM,Screen Size,Touch_Screen,Price
0,0,HP,15s-fq5007TU,Core i3,Windows 11 Home,512 GB,8 GB,39.62 cm (15.6 Inch),No,"₹38,990"
1,1,HP,15s-fy5003TU,Core i3,Windows 11 Home,512 GB,8 GB,39.62 cm (15.6 Inch),No,"₹37,990"
2,2,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"
3,3,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"
4,4,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8 GB,33.78 cm (13.3 inch),No,"₹70,990"


In [7]:
# Remove currency symbols and convert price to integers
laptop_data['Price'] = laptop_data['Price'].replace('[₹,]', '', regex=True).astype(int)

# Extracting numerical values from RAM
laptop_data['RAM'] = laptop_data['RAM'].str.extract('(\d+)').astype(int)

# Extracting screen size in inches
laptop_data['Screen Size'] = laptop_data['Screen Size'].str.extract('(\d+\.?\d+)').astype(float)

# Checking the modified data
laptop_data.head()


,Unnamed: 0,Brand,Model Name,Processor,Operating System,Storage,RAM,Screen Size,Touch_Screen,Price
0,0,HP,15s-fq5007TU,Core i3,Windows 11 Home,512 GB,8,39.62,No,38990
1,1,HP,15s-fy5003TU,Core i3,Windows 11 Home,512 GB,8,39.62,No,37990
2,2,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8,33.78,No,70990
3,3,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8,33.78,No,70990
4,4,Apple,2020 Macbook Air,M1,Mac OS Big Sur,256 GB,8,33.78,No,70990


In [8]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encoding of categorical features
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(laptop_data[['Brand', 'Processor', 'Operating System', 'Storage', 'Touch_Screen']])

# Creating a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Brand', 'Processor', 'Operating System', 'Storage', 'Touch_Screen']))

# Concatenate the encoded features with the numerical features
prepared_data = pd.concat([laptop_data[['RAM', 'Screen Size', 'Price']], encoded_df], axis=1)

# Display the prepared data
prepared_data.head()


,RAM,Screen Size,Price,Brand_ASUS,Brand_Acer,Brand_Apple,Brand_CHUWI,Brand_DELL,Brand_GIGABYTE,Brand_HP,...,Storage_2 TB,Storage_256 GB,Storage_3 TB,Storage_4 TB,Storage_512 GB,Storage_6 TB,Storage_64 GB,Storage_nan,Touch_Screen_No,Touch_Screen_Yes
0,8,39.62,38990,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,8,39.62,37990,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,8,33.78,70990,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,8,33.78,70990,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,8,33.78,70990,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Preparing the data
features = prepared_data.drop('Price', axis=1)
target = prepared_data['Price']

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Creating the Linear Regression model
linear_model = LinearRegression()

# Training the model
linear_model.fit(X_train, y_train)

# Predicting the test set
y_pred = linear_model.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2


(4277467097749.4673, -1.8104460979315318e+18)

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np

# Feature scaling
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Scale only the numerical features (we assume these are the first two columns: 'RAM' and 'Screen Size')
X_train_scaled[['RAM', 'Screen Size']] = scaler.fit_transform(X_train[['RAM', 'Screen Size']])
X_test_scaled[['RAM', 'Screen Size']] = scaler.transform(X_test[['RAM', 'Screen Size']])

# Checking for outliers in the Price column using the IQR method
Q1 = y_train.quantile(0.25)
Q3 = y_train.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering the training set to remove outliers
outlier_filter = (y_train >= lower_bound) & (y_train <= upper_bound)
X_train_filtered = X_train_scaled[outlier_filter]
y_train_filtered = y_train[outlier_filter]

# Creating and training a Ridge Regression model to include regularization
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_filtered, y_train_filtered)

# Predicting the test set
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Evaluating the model
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

mae_ridge, r2_ridge


(7513.759378143869, 0.6322292917695789)

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Initialize models
linear_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
decision_tree_model = DecisionTreeRegressor(random_state=42)
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)

# List of models for easy iteration
models = [
    ("Linear Regression", linear_model),
    ("Ridge Regression", ridge_model),
    ("Decision Tree", decision_tree_model),
    ("Random Forest", random_forest_model)
]

# Dictionary to hold model performance results
model_results = {}

# Training and evaluating each model
for name, model in models:
    model.fit(X_train_filtered, y_train_filtered)  # Using filtered data to train
    y_pred = model.predict(X_test_scaled)  # Predicting using scaled test data
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    model_results[name] = (mae, r2)

model_results


{'Linear Regression': (6997463349442478.0, -3.338374501223216e+24),
 'Ridge Regression': (7513.759378143869, 0.6322292917695789),
 'Decision Tree': (7645.901385602134, 0.5615061624687725),
 'Random Forest': (6969.192775249037, 0.6196421782441375)}

In [12]:
# Manually setting parameters for comparison
models_to_compare = [
    RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42),
    RandomForestRegressor(n_estimators=150, max_depth=None, random_state=42),
    RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42),
    RandomForestRegressor(n_estimators=150, max_depth=20, random_state=42)
]

# Dictionary to hold the performance of these models
comparison_results = {}

# Evaluating each configuration
for i, model in enumerate(models_to_compare):
    model.fit(X_train_filtered, y_train_filtered)
    y_pred_manual = model.predict(X_test_scaled)
    mae_manual = mean_absolute_error(y_test, y_pred_manual)
    r2_manual = r2_score(y_test, y_pred_manual)
    comparison_results[f"Model {i+1}"] = (mae_manual, r2_manual)

comparison_results


{'Model 1': (6969.192775249037, 0.6196421782441375),
 'Model 2': (7013.230446825527, 0.6146004524888729),
 'Model 3': (6955.513897248764, 0.6188843036435575),
 'Model 4': (7030.6413451724, 0.6132975872781583)}

In [13]:
# # # Prediction on New Data
# data_new = {'Brand': "HP", 'Model Name':"15s-fq5007TU", 'Processor': "Core i3", 'Operating System': "Windows 11 Home", 'Storage': "512 GB", 'RAM': 8,
#             'Screen Size': 39.62, 'Touch_Screen': "No"}
# index = [1]  # serial number
# new_data_df = pd.DataFrame(data_new, index)

In [14]:
import pickle

# Assuming 'encoder', 'scaler', and 'random_forest_model' are already fitted
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(random_forest_model, f)


In [15]:
# import pickle

# # Load the fitted encoder, scaler, and model
# with open('encoder.pkl', 'rb') as f:
#     encoder = pickle.load(f)
# with open('scaler.pkl', 'rb') as f:
#     scaler = pickle.load(f)
# with open('random_forest_model.pkl', 'rb') as f:
#     random_forest_model = pickle.load(f)

# # Now, proceed with the prediction steps as mentioned before


In [16]:
# import pandas as pd

# # New data to predict
# data_new = {
#     'Brand': ["HP"], 
#     'Processor': ["Core i3"], 
#     'Operating System': ["Windows 11 Home"], 
#     'Storage': ["512 GB"], 
#     'RAM': [8],
#     'Screen Size': [39.62], 
#     'Touch_Screen': ["No"]
# }
# new_data_df = pd.DataFrame(data_new)

# new_data_df



In [17]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

# Load the encoder, scaler, and model
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('random_forest_model.pkl', 'rb') as f:
    model = pickle.load(f)

# New data for prediction
data_new = {
    'Brand': ["HP"], 
    'Processor': ["Core i3"], 
    'Operating System': ["Windows 11 Home"], 
    'Storage': ["512 GB"], 
    'RAM': [8],
    'Screen Size': [39.62], 
    'Touch_Screen': ["No"]
}
new_data_df = pd.DataFrame(data_new)

# Encoding and scaling new data
encoded_new_data = encoder.transform(new_data_df[['Brand', 'Processor', 'Operating System', 'Storage', 'Touch_Screen']])
scaled_new_data = scaler.transform(new_data_df[['RAM', 'Screen Size']])

# Concatenate processed new data for prediction
final_new_data = np.hstack((encoded_new_data, scaled_new_data))

# Predict the price
predicted_price = model.predict(final_new_data)
print("Predicted Price:", predicted_price)


Predicted Price: [31448.315]


In [33]:
final_new_data

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1. 